In [1]:
'''
!!!ATENÇÃO!!!

ANTES DE EXECUTAR VERIFICAR A INSTALAÇÃO DAS BIBLIOTECAS
'''

#IMPORTS

import pandas as pd
import os
import shutil
import csv
import keyboard
from datetime import datetime
from time import sleep
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

#FUNÇÕES

#GERA AS NOTAS
def geraNotas(filename,pedido,cardapio,garcons):
    total = 0
    data = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    dataPath = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    with open ('caixa/'+str(dataPath)+'_'+str(filename), 'w', newline = '', encoding = 'UTF-8') as csv_file:
        writer = csv.writer(csv_file, delimiter = '║')
        writer.writerow(['╔' + '=' * 78 + '╗'])
        writer.writerow(['',' ' * 33 + 'CUPOM FISCAL' + ' ' * 33,''])
        writer.writerow(['','=' * 78,''])
        writer.writerow([''," " * 29 + str(data) + ' ' * 30,''])
        writer.writerow(['','=' * 78,''])
        writer.writerow(['','  QTDE  ',' ' * 17 +'PRODUTO'+' ' * 16, '  PREÇO  ','    TOTAL UNIT.   ',''])
        writer.writerow(['','=' * 78,''])
        for i in range(0,len(pedido)):
            produto = cardapio.loc[cardapio.COD_PRODUTO == pedido.loc[i,'CodItem'], 'PRODUTO'].item()
            preco = cardapio.loc[cardapio.COD_PRODUTO == pedido.loc[i,'CodItem'],'PREÇO'].item()
            quantidade = pedido.loc[i,'QtdItem'].item()
            totalUnitario = preco * pedido.loc[i,'QtdItem'].item()
            total += totalUnitario
            writer.writerow(['',' ' * 2 + str(quantidade) + ' ' * (6 - len(str(quantidade))),' ' + str(produto) + ' ' * (39 - len(produto)),' ' * 3 + str(preco) + ' ' * (6 - len(str(preco))),' ' * 6 + str(totalUnitario) + ' ' * (12 - len(str(totalUnitario))),''])           
            writer.writerow(['','=' * 78,''])
        writer.writerow(['',' ' * 27 + 'TOTAL' + ' ' * 27,' ' * 2  + 'R$' + '  ' + str(total) + ' ' * (12 - len(str(total))),''])
        writer.writerow(['╚' + '=' * 78 + '╝'])       
        
    with open ('faturamento/geral.csv', 'a', newline = '', encoding = 'UTF-8') as csv_file:
        writer = csv.writer(csv_file, delimiter = '|')
        for i in range(0,len(pedido)):
            produto = cardapio.loc[cardapio.COD_PRODUTO == pedido.loc[i,'CodItem'], 'PRODUTO'].item()
            quantidade = pedido.loc[i,'QtdItem'].item()
            preco = cardapio.loc[cardapio.COD_PRODUTO == pedido.loc[i,'CodItem'],'PREÇO'].item()
            totalUnitario = quantidade * preco
            garcom = garcons.loc[garcons.cod_garçom == pedido.loc[i,'CPF'],'nome_garçom'].item()
            dia = datetime.now().strftime('%d')
            mes = datetime.now().strftime('%m')
            ano = datetime.now().strftime('%Y')
            writer.writerow([produto,quantidade,preco,totalUnitario,garcom,dia,mes,ano])

#GERA AS ESTATISTICAS
def estatisticasDiarias():
    diaAtual = int(datetime.now().strftime('%d'))
    data = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    dataPath = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    dtf = pd.read_csv('faturamento/geral.csv', sep='|', names=['Produto','Qtde','Preco','totalUnitario','Garcom','Dia','Mes','Ano'])
    totGarcPorDia = dtf.groupby(['Dia','Garcom'])['totalUnitario'].sum()
    faturamentoPorDia = dtf.groupby(['Dia'])['totalUnitario'].sum()
    top3Garc = totGarcPorDia.groupby(['Dia']).nlargest(3)
    fatDia = str(faturamentoPorDia[diaAtual])
    topDiario = top3Garc[diaAtual]
    qtdeAtendimentos = dtf.groupby(['Dia'])['Garcom'].value_counts()
    Produtos = dtf.groupby(['Dia','Produto'])['Qtde'].sum()
    top3Prod = Produtos.groupby(['Dia']).nlargest(3)
    topDiarioProd = top3Prod[diaAtual]
    nome, val, nom, qt, nProd, qtProd, nmProd, qtdProd = [], [], [], [], [], [], [], []
    for i in range(0,3):
        nome.append(topDiario.index[i][1])
        nom.append(qtdeAtendimentos.index[i][1])
        nmProd.append(topDiarioProd.index[i][1])
    for i in topDiario:
        val.append(str(i))
    for i in qtdeAtendimentos:
        qt.append(str(i))
    for i in top3Prod:
        qtdProd.append(str(i))
    for i in range(0,Produtos.size):
        nProd.append(Produtos.index[i][1])
    for i in Produtos:
        qtProd.append(str(i))
    
    with open ('faturamento/'+'estatisticas_'+str(dataPath)+'.csv', 'w', newline = '', encoding = 'UTF-8') as csv_file:
        writer = csv.writer(csv_file, delimiter = '║')
        writer.writerow(['╔' + '=' * 78 + '╗'])
        writer.writerow(['',' ' * 30 + 'RESULTADOS DO DIA' + ' ' * 31,''])
        writer.writerow(['','=' * 78,''])
        writer.writerow([''," " * 29 + str(data) + ' ' * 30,''])
        writer.writerow(['','=' * 78,''])
        writer.writerow(['',(' ' * 5) +'PRODUTOS' + ' ' * 27, ' ' * 5 + 'QUANTIDADE VENDIDA' + ' ' * 14,''])
        writer.writerow(['','=' * 78,''])
        for i in range(0,len(nProd)):
            writer.writerow(['',(' ' * 5) + nProd[i] + ' ' * (35 - len(nProd[i])), (' ' * 5) + qtProd[i] + (32 - len(qtProd[i])) * ' ',''])
            writer.writerow(['','=' * 78,''])
        writer.writerow(['',(' ' * 5) +'TOP 3 PRODUTOS' + ' ' * 21, ' ' * 5 + 'QUANTIDADE VENDIDA' + ' ' * 14,''])
        writer.writerow(['','=' * 78,''])
        for i in range(0,3):
            writer.writerow(['',(' ' * 5) + nmProd[i] + ' ' * (35 - len(nmProd[i])), (' ' * 5) + qtdProd[i] + (32 - len(qtdProd[i])) * ' ',''])
            writer.writerow(['','=' * 78,''])
        writer.writerow(['',(' ' * 5) + 'FATURAMENTO TOTAL DO DIA' + ' ' * 11,' ' * 6 + 'R$' + '  ' + fatDia + (27 - len(fatDia)) * ' ',''])
        writer.writerow(['╚' + '=' * 78 + '╝'])
        
    with open ('ranking_garcom/'+'top_garcons_'+str(dataPath)+'.csv', 'w', newline = '', encoding = 'UTF-8') as csv_file:
        writer = csv.writer(csv_file, delimiter = '║')
        writer.writerow(['╔' + '=' * 78 + '╗'])
        writer.writerow(['',' ' * 30 + 'RESULTADOS DO DIA' + ' ' * 31,''])
        writer.writerow(['','=' * 78,''])
        writer.writerow([''," " * 29 + str(data) + ' ' * 30,''])
        writer.writerow(['','=' * 78,''])
        writer.writerow(['',(' ' * 5) + 'TOP 3 GARÇONS' + ' ' * 22, ' ' * 5 + 'TOTAL EM VENDAS' + ' ' * 17,''])
        writer.writerow(['','=' * 78,''])
        for i in range(0,3):
            writer.writerow(['',(' ' * 5) + nome[i] + ' ' * (35 - len(nome[i])), (' ' * 5) + 'R$  ' + val[i] + (28 - len(val[i])) * ' ',''])
            writer.writerow(['','=' * 78,''])
        writer.writerow(['',(' ' * 5) + 'TOP 3 GARÇONS' + ' ' * 22, ' ' * 5 + 'TOTAL EM ATENDIMENTOS' + ' ' * 11,''])
        for i in range(0,3):
            writer.writerow(['','=' * 78,''])
            writer.writerow(['',(' ' * 5) + nom[i] + ' ' * (35 - len(nom[i])), (' ' * 5) + qt[i] + (32 - len(qt[i])) * ' ',''])
        writer.writerow(['╚' + '=' * 78 + '╝'])
        
#ATENDE OS PEDIDOS
def atendePedido(filename):
    cardapio = pd.read_csv('dados/cardapio.csv',delimiter='|')
    garcons = pd.read_csv('dados/garcons.csv',delimiter='|')
    pedido = pd.read_csv('pedidos/'+str(filename),delimiter='\t',names=["CPF", "QtdItem", "CodItem"])
    geraNotas(filename,pedido,cardapio,garcons)
    data = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    shutil.move('pedidos/'+str(filename),'backup_pedidos/BKP_'+str(data)+'_'+str(filename))
    
#CRIA O WATCHDOG
class CreatedHandler(FileSystemEventHandler):
    def on_created(self, event):
        filepath, ext = os.path.splitext(event.src_path)
        filename = os.path.basename(event.src_path)
        if (ext == '.csv'):
            sleep(1)
            atendePedido(filename)
        if 'fim.txt' in os.listdir('pedidos') and len(os.listdir('pedidos')) == 2:
            estatisticasDiarias()
            print('--- FIM DO EXPEDIENTE ---')
            keyboard.press_and_release('I,I')
            
#################################################################################################

def main():
    event_handler = CreatedHandler()
    observer = Observer()
    observer.schedule(event_handler, path = 'pedidos/', recursive=False)
    observer.start()
    
    try:
        while True:
            sleep(1)
    except KeyboardInterrupt:
        os.remove('pedidos/fim.txt')
        observer.stop()
    observer.join()
    
#################################################################################################


In [2]:
main()

--- FIM DO EXPEDIENTE ---
